In [1]:
import pandas as pd
import numpy as np
import talib as ta

In [2]:
df_train = pd.read_feather('./data'+'/new_data.ftr')
df_train.set_index(["timestamp",'Asset_ID'], inplace=True)
df_train

Count          Open          High           Low  \
timestamp  Asset_ID                                                    
1514764860 2          40.0   2376.580000   2399.500000   2357.140000   
           0           5.0      8.530000      8.530000      8.530000   
           1         229.0  13835.194000  14013.800000  13666.110000   
           5          32.0      7.659600      7.659600      7.656700   
           7           5.0     25.920000     25.920000     25.874000   
...                    ...           ...           ...           ...   
1641772800 9         218.0    130.842571    130.940000    130.478000   
           10         20.0   2122.416667   2124.010000   2116.950000   
           13        118.0      0.066109      0.066130      0.066058   
           12        112.0      0.261055      0.261185      0.260474   
           11         52.0    190.732000    191.160000    189.800000   

                            Close        Volume          VWAP    Target  \
timestamp  Asset_ID                                                       
1514764860 2          2374.590000  1.923301e+01   2373.116392 -0.004218   
           0             8.530000  7.838000e+01      8.530000 -0.014399   
           1         13850.176000  3.155006e+01  13827.062093 -0.014643   
           5             7.657600  6.626713e+03      7.657713 -0.013922   
           7            25.877000  1.210873e+02     25.891363 -0.008264   
...                           ...           ...           ...       ...   
1641772800 9           130.707429  1.011042e+03    130.734076       NaN   
           10         2119.743333  2.084755e+00   2120.570582       NaN   
           13            0.066082  1.265238e+06      0.066086       NaN   
           12            0.260682  1.187581e+05      0.260913       NaN   
           11          190.506000  1.355932e+02    190.622811       NaN   

                       Weight     lr_15  Mkt_lrt_15  Crypto_Index  
timestamp  Asset_ID                                                
1514764860 2         0.058657       NaN    0.000000   2571.477256  
           0         0.105286       NaN    0.000000   2571.477256  
           1         0.165850       NaN    0.000000   2571.477256  
           5         0.033911       NaN    0.000000   2571.477256  
           7         0.050867       NaN    0.000000   2571.477256  
...                       ...       ...         ...           ...  
1641772800 9         0.058657 -0.004696   -0.003263   7532.261935  
           10        0.026874 -0.003462   -0.003263   7532.261935  
           13        0.043830 -0.003493   -0.003263   7532.261935  
           12        0.050867 -0.003568   -0.003263   7532.261935  
           11        0.039370 -0.000454   -0.003263   7532.261935  

[26473302 rows x 12 columns]

Weighted info `lr_mkt_resid` similar to `Target` which can remove the market signal from individual asset returns. 

$$R^{a}(t)=\log P^{a}_t - \log P^{a}_{t-15}$$

$$M(t)=\frac{\sum_{a} w^{a} R^{a}(t)}{\sum_{a} w^{a}}$$

$$\beta^{a}=\frac{\left\langle M \cdot R^{a}\right\rangle_{60}}{\left\langle M^{2}\right\rangle_{60}}$$

$$lrmktresid^{a}(t)=R^{a}(t)-\beta^{a} M(t)$$

where the bracket  $⟨.⟩_{60}$ represent the rolling average over time (60 minute windows).

In [3]:
def beta(df, window=60): 
    b = (ta.MULT(df.Mkt_lrt_15,df.lr_15).rolling(window).mean())/(ta.MULT(df.Mkt_lrt_15,df.Mkt_lrt_15).rolling(window).mean())
    return b

def lr_mkt_resid(df):
    return ta.SUB(df.lr_15, ta.MULT(df.beta, df.Mkt_lrt_15))

def rolling_volatility(df, window=60):
    return df.rolling(window).std()

def lrtn_index(crypto_index):
    lrtn_index_5 = np.log(crypto_index).diff(5).rename('lrtn_index_5')
    lrtn_index_15 = np.log(crypto_index).diff(15).rename('lrtn_index_15')
    return pd.concat([lrtn_index_5,lrtn_index_15],axis=1)

def lr_mkt_resid_sma(lr_mkt_resid):
    return ta.SMA(lr_mkt_resid,5)
#############################################
def lambda_by_Asset(x):
    ##vol_sum_15
    vol_sum_15 = (ta.SMA(x['Volume'],15)*15).rename('vol_sum_15')
    ##std
    stds = rolling_volatility(x[['lr_15','Mkt_lrt_15','Crypto_Index']])
    stds.columns = ['std_lr_15','std_Mkt_lrt_15','std_Crypto_Index']
    ##lrtn_index
    lrtn_index_5_15 = lrtn_index(x['Crypto_Index'])
    ##beta
    b = beta(x).rename("beta")#.to_frame()#.reset_index(0,drop=True)
    b = b.replace([np.nan,np.inf,-np.inf], 0)
    xx = pd.concat([vol_sum_15, stds,lrtn_index_5_15,b], axis=1)
    x= x.merge(xx, on =['timestamp','Asset_ID'],how='left')
    ##
    x['lr_mkt_resid'] = lr_mkt_resid(x)
    ##
    lr_mkt_resid_sma_5 = lr_mkt_resid_sma(x['lr_mkt_resid']).rename("lr_mkt_resid_sma")
    return pd.concat([x,lr_mkt_resid_sma_5], axis=1)

In [4]:
#######################################add features by asset_id
df_train = df_train.groupby('Asset_ID').apply(lambda x: lambda_by_Asset(x))
df_train.iloc[-14:,]


Count          Open          High           Low  \
timestamp  Asset_ID                                                     
1641772800 3          351.0      1.170103      1.170729      1.167000   
           2          106.0    376.696667    376.950000    376.000000   
           0          288.0    438.722333    438.800000    437.700000   
           1         2111.0  41860.168571  41877.700000  41766.700000   
           4          197.0      0.150498      0.150600      0.150200   
           5           88.0      2.794480      2.800000      2.788562   
           7           75.0     29.731780     29.751000     29.698200   
           6         2059.0   3151.504286   3152.150000   3143.790000   
           8           29.0      1.034633      1.075000      0.957800   
           9          218.0    130.842571    130.940000    130.478000   
           10          20.0   2122.416667   2124.010000   2116.950000   
           13         118.0      0.066109      0.066130      0.066058   
           12         112.0      0.261055      0.261185      0.260474   
           11          52.0    190.732000    191.160000    189.800000   

                            Close        Volume          VWAP  Target  \
timestamp  Asset_ID                                                     
1641772800 3             1.167863  1.692235e+05      1.168722     NaN   
           2           376.480000  1.443180e+02    376.560393     NaN   
           0           437.990333  4.016819e+02    438.278705     NaN   
           1         41823.085714  8.982209e+01  41830.240059     NaN   
           4             0.150307  1.145598e+06      0.150394     NaN   
           5             2.789640  2.411353e+04      2.792239     NaN   
           7            29.715100  1.063943e+03     29.721728     NaN   
           6          3147.158571  5.786272e+02   3148.128143     NaN   
           8             1.034400  8.585877e+03      1.034341     NaN   
           9           130.707429  1.011042e+03    130.734076     NaN   
           10         2119.743333  2.084755e+00   2120.570582     NaN   
           13            0.066082  1.265238e+06      0.066086     NaN   
           12            0.260682  1.187581e+05      0.260913     NaN   
           11          190.506000  1.355932e+02    190.622811     NaN   

                       Weight     lr_15  ...  Crypto_Index    vol_sum_15  \
timestamp  Asset_ID                      ...                               
1641772800 3         0.107797 -0.005190  ...   7532.261935  1.410050e+06   
           2         0.058657 -0.002764  ...   7532.261935  7.295325e+02   
           0         0.105286 -0.002281  ...   7532.261935  5.152490e+03   
           1         0.165850 -0.002324  ...   7532.261935  5.309589e+02   
           4         0.086971 -0.003500  ...   7532.261935  4.875971e+06   
           5         0.033911 -0.005577  ...   7532.261935  1.716800e+05   
           7         0.050867 -0.003969  ...   7532.261935  5.021665e+03   
           6         0.144188 -0.003110  ...   7532.261935  5.335395e+03   
           8         0.026874 -0.001899  ...   7532.261935  5.350608e+04   
           9         0.058657 -0.004696  ...   7532.261935  5.065137e+03   
           10        0.026874 -0.003462  ...   7532.261935  2.962160e+01   
           13        0.043830 -0.003493  ...   7532.261935  1.020347e+07   
           12        0.050867 -0.003568  ...   7532.261935  1.559958e+06   
           11        0.039370 -0.000454  ...   7532.261935  8.948188e+02   

                     std_lr_15  std_Mkt_lrt_15  std_Crypto_Index  \
timestamp  Asset_ID                                                
1641772800 3          0.003322        0.002552         20.825669   
           2          0.002486        0.002552         20.825669   
           0          0.002149        0.002552         20.825669   
           1          0.002688        0.002552         20.825669   
           4          0.002200        0.002552         20.8

In [5]:
df_train.reset_index().to_feather('./data/new_data.ftr')